In [35]:
import pandas as pd
import pymongo
from sqlalchemy import create_engine
from config import username
from config import password

ImportError: cannot import name 'username'

In [2]:
artists = pd.read_csv('artists.csv')
display(artists.info())
artists.head()

C:\Users\jewel\AppData\Local\Continuum\anaconda3\envs\PythonData\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (2,4,6) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1466083 entries, 0 to 1466082
Data columns (total 10 columns):
 #   Column            Non-Null Count    Dtype  
---  ------            --------------    -----  
 0   mbid              1466083 non-null  object 
 1   artist_mb         1466075 non-null  object 
 2   artist_lastfm     986756 non-null   object 
 3   country_mb        662368 non-null   object 
 4   country_lastfm    211498 non-null   object 
 5   tags_mb           119946 non-null   object 
 6   tags_lastfm       381075 non-null   object 
 7   listeners_lastfm  986760 non-null   float64
 8   scrobbles_lastfm  986760 non-null   float64
 9   ambiguous_artist  1466083 non-null  bool   
dtypes: bool(1), float64(2), object(7)
memory usage: 102.1+ MB


None

,mbid,artist_mb,artist_lastfm,country_mb,country_lastfm,tags_mb,tags_lastfm,listeners_lastfm,scrobbles_lastfm,ambiguous_artist
0,cc197bad-dc9c-440d-a5b5-d52ba2e14234,Coldplay,Coldplay,United Kingdom,United Kingdom,rock; pop; alternative rock; british; uk; brit...,rock; alternative; britpop; alternative rock; ...,5381567.0,360111850.0,False
1,a74b1b7f-71a5-4011-9441-d0b5e4122711,Radiohead,Radiohead,United Kingdom,United Kingdom,rock; electronic; alternative rock; british; g...,alternative; alternative rock; rock; indie; el...,4732528.0,499548797.0,False
2,8bfac288-ccc5-448d-9573-c33ea2aa5c30,Red Hot Chili Peppers,Red Hot Chili Peppers,United States,United States,rock; alternative rock; 80s; 90s; rap; metal; ...,rock; alternative rock; alternative; Funk Rock...,4620835.0,293784041.0,False
3,73e5e69d-3554-40d8-8516-00cb38737a1c,Rihanna,Rihanna,United States,Barbados; United States,pop; dance; hip hop; reggae; contemporary r b;...,pop; rnb; female vocalists; dance; Hip-Hop; Ri...,4558193.0,199248986.0,False
4,b95ce3ff-3d05-4e87-9e01-c97b66af13d4,Eminem,Eminem,United States,United States,turkish; rap; american; hip-hop; hip hop; hiph...,rap; Hip-Hop; Eminem; hip hop; pop; american; ...,4517997.0,199507511.0,False


In [3]:
new_artists = artists[['artist_lastfm', 'country_lastfm', 'tags_lastfm', 'listeners_lastfm', 'scrobbles_lastfm']]
new_artists.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1466083 entries, 0 to 1466082
Data columns (total 5 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   artist_lastfm     986756 non-null  object 
 1   country_lastfm    211498 non-null  object 
 2   tags_lastfm       381075 non-null  object 
 3   listeners_lastfm  986760 non-null  float64
 4   scrobbles_lastfm  986760 non-null  float64
dtypes: float64(2), object(3)
memory usage: 55.9+ MB


In [4]:
clean_artists = new_artists.dropna(how= 'any')
clean_artists.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 204418 entries, 0 to 986728
Data columns (total 5 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   artist_lastfm     204418 non-null  object 
 1   country_lastfm    204418 non-null  object 
 2   tags_lastfm       204418 non-null  object 
 3   listeners_lastfm  204418 non-null  float64
 4   scrobbles_lastfm  204418 non-null  float64
dtypes: float64(2), object(3)
memory usage: 9.4+ MB


In [5]:
clean_artists = clean_artists.rename(columns= {'artist_lastfm': 'Artist', 'country_lastfm': 'Country', 'tags_lastfm': 'Genre', 'listeners_lastfm': 'Listener Count', 'scrobbles_lastfm': 'Play Count'})

In [6]:
clean_artists['Genre'] = clean_artists['Genre'].str.replace('seen live;', '')

In [7]:
clean_artists['Genre'] = clean_artists['Genre'].str.split(';', expand= True)[0]
clean_artists.head()

,Artist,Country,Genre,Listener Count,Play Count
0,Coldplay,United Kingdom,rock,5381567.0,360111850.0
1,Radiohead,United Kingdom,alternative,4732528.0,499548797.0
2,Red Hot Chili Peppers,United States,rock,4620835.0,293784041.0
3,Rihanna,Barbados; United States,pop,4558193.0,199248986.0
4,Eminem,United States,rap,4517997.0,199507511.0


In [8]:
clean_artists.drop_duplicates(subset= 'Artist', keep = 'first', inplace = True)


In [9]:
clean_artists['Artist'].value_counts()

Club Dogo               1
Κεφάλαιο 24             1
MORKHGRAT               1
Gazde                   1
Bellicist               1
                       ..
The Strawberry Smell    1
Amadodana Ase Wesile    1
Matthew Barber          1
Tim Berg                1
Cyklon B                1
Name: Artist, Length: 181533, dtype: int64

In [10]:
clean_artists['Genre'].value_counts()

black metal        5105
electronic         4717
folk               4150
jazz               4134
pop                4038
                   ... 
colombian rock        1
malta                 1
karaoke               1
 coban rock           1
australian punk       1
Name: Genre, Length: 6002, dtype: int64

In [11]:
clean_artist = clean_artists.groupby(['Genre'])
clean_artist.count()

,Artist,Country,Listener Count,Play Count
Genre,,,,
austrian,1,1,1,1
2step,1,1,1,1
60s,1,1,1,1
70s,3,3,3,3
8-bit,1,1,1,1
...,...,...,...,...
zouglou,2,2,2,2
zouk,53,53,53,53
zuo,1,1,1,1


In [12]:
#clean_artists['Counts'] = clean_artists.groupby(['Genre'])(['Artist']).transform('count')

In [13]:
z = clean_artists['Genre'].value_counts()

z1 = z.to_dict()

clean_artists['Counts'] = clean_artists['Genre'].map(z1)

clean_artists.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 181533 entries, 0 to 986728
Data columns (total 6 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   Artist          181533 non-null  object 
 1   Country         181533 non-null  object 
 2   Genre           181533 non-null  object 
 3   Listener Count  181533 non-null  float64
 4   Play Count      181533 non-null  float64
 5   Counts          181533 non-null  int64  
dtypes: float64(2), int64(1), object(3)
memory usage: 9.7+ MB


In [14]:
clean_artists = clean_artists.loc[clean_artists['Counts'] >= 750]

In [15]:
clean_artists['Genre'].value_counts()

black metal            5105
electronic             4717
folk                   4150
jazz                   4134
pop                    4038
rock                   3970
japanese               3766
indie                  3672
Hip-Hop                3487
punk                   3411
hardcore               2330
death metal            2191
ambient                2111
french                 2045
Progressive rock       1856
heavy metal            1711
Classical              1664
female vocalists       1630
Soundtrack             1548
trance                 1525
reggae                 1516
thrash metal           1448
experimental           1408
swedish                1279
post-rock              1232
House                  1158
hard rock              1105
Drum and bass          1054
j-pop                  1051
soul                   1020
blues                  1006
singer-songwriter       997
finnish                 991
post-punk               987
doom metal              978
rap                 

In [16]:
del clean_artists['Counts']
clean_artists.head()

,Artist,Country,Genre,Listener Count,Play Count
0,Coldplay,United Kingdom,rock,5381567.0,360111850.0
1,Radiohead,United Kingdom,alternative,4732528.0,499548797.0
2,Red Hot Chili Peppers,United States,rock,4620835.0,293784041.0
3,Rihanna,Barbados; United States,pop,4558193.0,199248986.0
4,Eminem,United States,rap,4517997.0,199507511.0


In [19]:
billboard = pd.read_csv('Billboard.csv')
display(billboard.info())
billboard.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1951 entries, 0 to 1950
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Unnamed: 0      1951 non-null   int64 
 1   Song            1951 non-null   object
 2   Artist          1951 non-null   object
 3   Weeks On #1     1951 non-null   int64 
 4   Weeks On Chart  1951 non-null   int64 
 5   Peak Rank       1951 non-null   int64 
dtypes: int64(4), object(2)
memory usage: 91.6+ KB


None

,Unnamed: 0,Song,Artist,Weeks On #1,Weeks On Chart,Peak Rank
0,0,Blank Space,Taylor Swift,7,36,1
1,1,Take Me To Church,Hozier,0,41,2
2,2,Uptown Funk!,Mark Ronson Featuring Bruno Mars,14,56,1
3,3,Thinking Out Loud,Ed Sheeran,0,58,2
4,4,Lips Are Movin,Meghan Trainor,0,29,4


In [20]:
new_billboard = billboard[['Artist', 'Weeks On #1', 'Weeks On Chart']]
new_billboard.head()

,Artist,Weeks On #1,Weeks On Chart
0,Taylor Swift,7,36
1,Hozier,0,41
2,Mark Ronson Featuring Bruno Mars,14,56
3,Ed Sheeran,0,58
4,Meghan Trainor,0,29


In [21]:
new_billboard['Artist'].value_counts()

Drake                                              64
The Weeknd                                         26
Future                                             25
J. Cole                                            25
XXXTENTACION                                       22
                                                   ..
Juice WRLD Featuring Lil Uzi Vert                   1
Eazy-E                                              1
Jimmie Allen                                        1
Empire Cast Featuring Estelle & Jussie Smollett     1
Khalid & Swae Lee                                   1
Name: Artist, Length: 1072, dtype: int64

In [22]:
new_billboard[['Artist']] = new_billboard.Artist.str.split(" Featuring", expand= True)[0]
new_billboard.head()

C:\Users\jewel\AppData\Local\Continuum\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\jewel\AppData\Local\Continuum\anaconda3\envs\PythonData\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,Artist,Weeks On #1,Weeks On Chart
0,Taylor Swift,7,36
1,Hozier,0,41
2,Mark Ronson,14,56
3,Ed Sheeran,0,58
4,Meghan Trainor,0,29


In [23]:
new_billboard[['Artist']] = new_billboard.Artist.str.split(" &", expand= True)[0]
new_billboard.head(15)

C:\Users\jewel\AppData\Local\Continuum\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\jewel\AppData\Local\Continuum\anaconda3\envs\PythonData\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,Artist,Weeks On #1,Weeks On Chart
0,Taylor Swift,7,36
1,Hozier,0,41
2,Mark Ronson,14,56
3,Ed Sheeran,0,58
4,Meghan Trainor,0,29
5,Sam Smith,0,37
6,Ariana Grande,0,22
7,Nick Jonas,0,32
8,Maroon 5,0,33
9,Meghan Trainor,0,47


In [24]:
new_billboard[['Artist']] = new_billboard.Artist.str.split(" x", expand= True)[0]
new_billboard.head(15)

C:\Users\jewel\AppData\Local\Continuum\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\jewel\AppData\Local\Continuum\anaconda3\envs\PythonData\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,Artist,Weeks On #1,Weeks On Chart
0,Taylor Swift,7,36
1,Hozier,0,41
2,Mark Ronson,14,56
3,Ed Sheeran,0,58
4,Meghan Trainor,0,29
5,Sam Smith,0,37
6,Ariana Grande,0,22
7,Nick Jonas,0,32
8,Maroon 5,0,33
9,Meghan Trainor,0,47


In [25]:
new_billboard[['Artist']] = new_billboard.Artist.str.split(",", expand= True)[0]
new_billboard.head(15)

C:\Users\jewel\AppData\Local\Continuum\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\jewel\AppData\Local\Continuum\anaconda3\envs\PythonData\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,Artist,Weeks On #1,Weeks On Chart
0,Taylor Swift,7,36
1,Hozier,0,41
2,Mark Ronson,14,56
3,Ed Sheeran,0,58
4,Meghan Trainor,0,29
5,Sam Smith,0,37
6,Ariana Grande,0,22
7,Nick Jonas,0,32
8,Maroon 5,0,33
9,Meghan Trainor,0,47


In [26]:
new_billboard['Artist'].value_counts()

Drake                93
Future               39
The Weeknd           37
Meek Mill            36
Kendrick Lamar       26
                     ..
Ava Max               1
Casper Magico         1
Tyler Farr            1
Empire Of The Sun     1
Mike WiLL Made-It     1
Name: Artist, Length: 570, dtype: int64

In [46]:
new_billboard = new_billboard.loc[new_billboard['Artist'] != '\r\r\r\r\n\r\r\r\r\nAriana Grande\r\r\r\r\n\r\r\r\r\n']
new_billboard.set_index('Artist')

,Weeks On #1,Weeks On Chart
Artist,,
Taylor Swift,7,36
Hozier,0,41
Mark Ronson,14,56
Ed Sheeran,0,58
Meghan Trainor,0,29
...,...,...
Cody Johnson,0,0
YNW Melly,0,0
Maren Morris,0,0


In [48]:
billboard_group = new_billboard.groupby('Artist').sum()

billboard_group.info()


<class 'pandas.core.frame.DataFrame'>
Index: 569 entries, 2 Chainz to twenty one pilots
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype
---  ------          --------------  -----
 0   Weeks On #1     569 non-null    int64
 1   Weeks On Chart  569 non-null    int64
dtypes: int64(2)
memory usage: 13.3+ KB


In [29]:
combined_tables = pd.merge(clean_artists, billboard_group, how= 'inner', on = 'Artist')
combined_tables.head()

,Artist,Country,Genre,Listener Count,Play Count,Weeks On #1,Weeks On Chart
0,Coldplay,United Kingdom,rock,5381567.0,360111850.0,0,43
1,Red Hot Chili Peppers,United States,rock,4620835.0,293784041.0,0,2
2,Rihanna,Barbados; United States,pop,4558193.0,199248986.0,9,186
3,Eminem,United States,rap,4517997.0,199507511.0,0,60
4,Kanye West,United States,Hip-Hop,4390502.0,238603850.0,0,114


In [40]:
rds_connection_string = f"<username>:<password>@localhost:5432/artist_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [41]:
engine.table_names()

['clean_artist', 'billboard']

In [42]:
clean_artists.to_sql(name='clean_artist', con=engine, if_exists='append', index=False)

In [43]:
billboard_group.to_sql(name='billboard', con=engine, if_exists='append', index=False)

IntegrityError: (psycopg2.errors.NotNullViolation) null value in column "Artist" violates not-null constraint
DETAIL:  Failing row contains (null, 0, 63).

[SQL: INSERT INTO billboard ("Weeks On #1", "Weeks On Chart") VALUES (%(Weeks On #1)s, %(Weeks On Chart)s)]
[parameters: ({'Weeks On #1': 0, 'Weeks On Chart': 63}, {'Weeks On #1': 0, 'Weeks On Chart': 76}, {'Weeks On #1': 0, 'Weeks On Chart': 43}, {'Weeks On #1': 0, 'Weeks On Chart': 8}, {'Weeks On #1': 0, 'Weeks On Chart': 100}, {'Weeks On #1': 0, 'Weeks On Chart': 59}, {'Weeks On #1': 0, 'Weeks On Chart': 2}, {'Weeks On #1': 0, 'Weeks On Chart': 20}  ... displaying 10 of 569 total bound parameter sets ...  {'Weeks On #1': 0, 'Weeks On Chart': 22}, {'Weeks On #1': 0, 'Weeks On Chart': 144})]
(Background on this error at: http://sqlalche.me/e/gkpj)